In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from rwr_functions import *
from constants import *
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
results = []
for netname in NETWORKS:
    graph = load_graph_nx(netname)
    
    
    for disease in tqdm(DISEASES):
        pegasus_data = load_pegasus_results(disease)
        
        gene_seeds_ncbi, ncbi_targets = load_seeds_and_targets(disease)
        ncbi_targets = list(set(ncbi_targets))

        score_mapping = {g: 1.0 if g in gene_seeds_ncbi else 0.0 for g in pegasus_data["NCBI_id"]}
        pegasus_data.loc[:, "Score"] = pegasus_data["NCBI_id"].map(score_mapping)
        pagerank_seeds = init_rwr_scores_nx(graph, pegasus_data)

        for alpha in tqdm(ALPHAS[1:]):
            rwr_scores = perform_rwr_nx(alpha, graph, pagerank_seeds)
            rwr_results = process_rwr_results_nx(rwr_scores, graph, pegasus_data, pagerank_seeds) 
            for K in Ks:
                metrics = calculate_metrics(rwr_results, K, gene_seeds_ncbi, ncbi_targets, netname, alpha, disease, scoring="Seed")
                results.append(metrics)

results = pd.concat(results)
results

In [ ]:
os.makedirs("outputs", exist_ok=True)
results.to_csv("outputs/gene_seeds_single_net_metrics.csv", index=False)